# Q1.
classifier for mnist dataset that achieves over 97% acccuracy

In [1]:
from sklearn.datasets import fetch_openml
mnist=fetch_openml('mnist_784',as_frame=False) #as_frame= false means numpy array

In [2]:
X,y=mnist.data,mnist.target

In [3]:
X.shape

(70000, 784)

In [4]:
X_train,X_test,y_train,y_test=X[:60000],X[60000:],y[:60000],y[60000:]

In [5]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train,y_train)
y_pred=knn_clf.predict(X_train)

In [6]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,y_pred) # train score is quite nice

0.9819166666666667

In [7]:
# lets see validation score using cross val score
from sklearn.model_selection import cross_val_score
cross_val_score(knn_clf,X_train,y_train,cv=3,scoring="accuracy")

array([0.9676 , 0.9671 , 0.96755])

using grid search to improve the accuracy

In [8]:
from sklearn.model_selection import GridSearchCV
param_grid=[{'n_neighbors':[4,6,8],'leaf_size':[32,34],'weights':['uniform','distance']}]
grid_search=GridSearchCV(knn_clf,param_grid,cv=3,scoring='accuracy')

In [9]:
grid_search.fit(X_train,y_train) 

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid=[{'leaf_size': [32, 34], 'n_neighbors': [4, 6, 8],
                          'weights': ['uniform', 'distance']}],
             scoring='accuracy')

In [10]:
grid_search.best_params_

{'leaf_size': 32, 'n_neighbors': 4, 'weights': 'distance'}

In [11]:
grid_search.best_score_

0.9703500000000002

In [13]:
grid_search.best_estimator_

KNeighborsClassifier(leaf_size=32, n_neighbors=4, weights='distance')

In [14]:
tuned_score=grid_search.score(X_test,y_test)

In [15]:
tuned_score

0.9714

##### goal achieved

# Q2.

Data Augmentation on mnist

In scipy.ndimage, the shift function is used to perform geometric translations on n-dimensional arrays, including images. It allows you to shift the elements of an input array along specified axes by a given number of pixels in each dimension.

In [24]:
from scipy.ndimage import shift
def shift_image(image,x,y):
    image=image.reshape(28,28)
    shift_image=shift(image,[x,y],cval=0.0,mode="constant")
    return shift_image.reshape([-1]) #flatten the image into 1D

In [25]:
direction=((-1,0),(0,1),(1,0),(0,-1))
X_train_augment=[image for image in X_train]
y_train_augment=[label for label in y_train]

for x,y in direction:
    for image,label in zip(X_train,y_train):
        X_train_augment.append(shift_image(image,x,y))
        y_train_augment.append(label)

In [26]:
import numpy as np
X_train_augment=np.array(X_train_augment)
y_train_augment=np.array(y_train_augment)

In [27]:
shuffle_idx=np.random.permutation(len(X_train_augment))
X_train_augment=X_train_augment[shuffle_idx]
y_train_augment=y_train_augment[shuffle_idx]

In [28]:
knn_clf=KNeighborsClassifier(**grid_search.best_params_)

In [30]:
X_train_augment.shape,y_train_augment.shape

((300000, 784), (300000,))

In [31]:
knn_clf.fit(X_train_augment,y_train_augment)

KNeighborsClassifier(leaf_size=32, n_neighbors=4, weights='distance')

In [32]:
augment_score=knn_clf.score(X_test,y_test)

In [33]:
augment_score # 0.9763-0.9714=0.0049 means 0.49% accuracy boost

0.9763

In [34]:
# lets look error rate
error_rate=(1-augment_score)/(1-tuned_score)-1
print(f"Error rate change:{error_rate:.0%}")

Error rate change:-17%


In [ ]:
Error rate change is -17% which is huge